$\bar{B}=\frac{B(S+2L_{x}\lambda)}{2f'(L_{x}/2)\int_{-L_{z}/2}^{L_{z}/2}g(z)dz}=\frac{BL_{x}(L_{z}/2+\lambda)}{f'(L_{x}/2)\int_{-L_{z}/2}^{L_{z}/2}g(z)dz}=C_{0}*B$ \
with $C_{0}=\frac{L_{x}(L_{z}/2+\lambda)}{f'(L_{x}/2)\int_{-L_{z}/2}^{L_{z}/2}g(z)dz}$

consider
$f(x)=\frac{1}{2}x^{2}$ and $g(z)={\rm{sech}}^{2}(\frac{z}{l})$\
then $C_{0}=\frac{(L_{z}/2+\lambda)}{l\tanh(\frac{L_{z}/2}{l})}$




In [8]:
%matplotlib
import kwant
from matplotlib import pyplot
import tinyarray
import numpy as np
from numpy import sqrt
import time
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from numpy import exp,pi,kron,cos,sin,sqrt,pi,cosh,tanh
from pathos.multiprocessing import Pool 
from scipy import sparse, integrate
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


sigma0 = np.array([[1,0],[0,1]])
sigmax = np.array([[0,1],[1,0]])
sigmay = np.array([[0,-1j],[1j,0]])
sigmaz = np.array([[1,0],[0,-1]])

tau0 = sigma0
taux = sigmax
tauy = sigmay
tauz = sigmaz

s0 = sigma0
sx = sigmax
sy = sigmay
sz = sigmaz


Lz =12
Lx=20
Ly=20

normal=4
Lsc=(Lz-normal)//2


M = -1.5
A1 = 2
A2 = 2
tx = 1
ty = 1
tz = 1
delta=0.5
mu=1*0


pen_len=Lz
decay_l=0.2
C0=((pen_len+(Lz//2))/decay_l)/tanh((Lz//2)/decay_l)



def kwant_square_band_TSC_G_Y(theta,phi):

    def sys_shape(pos):
        es = 1e-3
        x,z = pos
        return (-es-Lx//2 <x< Lx//2+es and -es-Lz//2< z <Lz//2+es )  
                 
    def lead_shape(pos):
        es = 1e-3
        x,z = pos
        return ( -es-Lx//2 <x< Lx//2+es and -es-Lz//2< z <Lz//2+es )  

    
    def onsite(site,ky,lam_ex,phi_sc):
        x,z = site.pos
        if z>=(Lz//2-Lsc):
            h_sc=kron(taux,kron(s0,sigma0))
            Lam_ex=0
        elif z<=-(Lz//2-Lsc):
            h_sc=cos(phi_sc)*kron(taux,kron(s0,sigma0))+sin(phi_sc)*kron(tauy,kron(s0,sigma0))
            Lam_ex=0
        else:
            h_sc=0*kron(taux,kron(s0,sigma0))
            Lam_ex=lam_ex
        onsite_e =((M+2*tx+2*ty*(1-cos(ky))+2*tz)*kron(s0,sigmaz)+A2*sin(ky)*kron(sy,sigmax)\
                   -mu*kron(s0,sigma0))

        zeeman= Lam_ex*(cos(theta)*sin(phi)*kron(sx,sigmax)+sin(theta)*sin(phi)*kron(sy,sigmax)\
                    +cos(phi)*kron(sz,sigmax))
        
        return kron(tauz,onsite_e)+ delta*h_sc+kron(tau0,zeeman)
                
    def hopping_x(site1,site2,eta,Bou_x):
        x1,z1 = site1.pos
        x2,z2 = site2.pos
        z=(z1+z2)/2
        hopx_e=(-tx*kron(s0,sigmaz)+A2/(2j)*kron(sx,sigmax))
        phase=eta*(z*pen_len/(Lz//2) + C0*(1/cosh(z/decay_l)**2)*tanh(z/decay_l)*\
                                              ((1/3)*(x2**3-x1**3) - Bou_x*(Lx//2)**2)/decay_l)
        
#         if z1<=-(Lz//2-Lsc) or z2<=-(Lz//2-Lsc):
#             return kron(tauz,hopx_e)
#         elif z1>=(Lz//2-Lsc) or z2>=(Lz//2-Lsc):
#             return kron(tauz,hopx_e)
#         else:
        return kron(tauz,hopx_e)*cos(2*pi*phase)+1j*kron(tau0,hopx_e)*sin(2*pi*phase)
                
    def hopping_z(site1,site2,eta):
        x1,z1 = site1.pos
        x2,z2 = site2.pos
        x=(x1+x2)/2
        hopz_e=-tz*kron(s0,sigmaz)+A1/(2j)*kron(sz,sigmax)
        phase=eta*x*C0*decay_l*(tanh(z2/decay_l)-tanh(z1/decay_l))
        
#         if z1<=-(Lz//2-Lsc) or z2<=-(Lz//2-Lsc):
#             return kron(tauz,hopz_e)
#         elif z1>=(Lz//2-Lsc) or z2>=(Lz//2-Lsc):
#             return kron(tauz,hopz_e)
#         else:
        return kron(tauz,hopz_e)*cos(2*pi*phase)+1j*kron(tau0,hopz_e)*sin(2*pi*phase)

    lat = kwant.lattice.square(1,norbs=8)
    sys = kwant.Builder()
    sys[lat.shape(sys_shape, (0, 0))] = onsite
    sys[kwant.builder.HoppingKind((1, 0), lat, lat)] = hopping_x
    sys[kwant.builder.HoppingKind((0,1), lat, lat)] = hopping_z

    return sys




z=np.linspace(-Lz//2,Lz//2,201)
Ax=(1/cosh(z/decay_l))**2

plt.figure()
plt.plot(z, Ax,'b')
plt.ylabel("sech$^{2}$(z/l)")
plt.xlabel("real-space:z")
plt.xticks([-Lz//2,-(Lz//2-Lsc),0,(Lz//2-Lsc),Lz//2])
plt.show




"""
With potential the phase different change to 
\phi_{sc} \pm 2*pi*\phi

\phi=2B(S+2*Lx*\lambda)/(h/e)=2*\eta*B(S+2*Lx*\lambda)

\eta is the flux in unit area

"""

B_plot=((2*pi)/(2*pi*2*(25)*(Lz+2*pen_len))*(4*1e-15)/(1e-9)**2)/2

del_phi=1*pi
eta=del_phi/(2*pi*2*(Lx//2)*(Lz+2*pen_len))#(0.1/((4*1e-15)/(1e-9)**2))

system_TSC_G_Y = kwant_square_band_TSC_G_Y(0,pi/2).finalized()


def cal_energy_TSC_G_Y(k,phi_sc):
    ham_mat = system_TSC_G_Y.hamiltonian_submatrix(sparse=True, params = dict(ky=k,lam_ex=0.*delta,Bou_x=0,\
                                                                              eta=eta,phi_sc=phi_sc))
    energies = sparse.linalg.eigsh(ham_mat, k=40,sigma=0,return_eigenvectors=False,which='LM')
    return np.sort(energies/delta)


ks=np.linspace(-0.5,0.5,21)*pi
Phi_sc=np.linspace(0.,2,51)*pi


# with Pool() as process_pool:
#     kwant_square_band_TSC_G_Y = process_pool.starmap_async(cal_energy_TSC_G_Y,zip(ks,pi*np.ones(len(ks)))).get()

# with Pool() as process_pool:
#     Eng_TSC_G_Y_pi = process_pool.starmap_async(cal_energy_TSC_G_Y,zip(ks,pi*np.ones(len(ks)))).get()
    
with Pool() as process_pool:
    Andreev_TSC_G_Y = process_pool.starmap_async(cal_energy_TSC_G_Y,zip(0*np.ones(len(Phi_sc)),Phi_sc)).get()

    
    
    
plt.figure()
plt.plot(ks/pi, np.array(Eng_TSC_G_Y_0),'b',lw=2)
pyplot.xticks(fontsize=15)
pyplot.yticks(fontsize=15)
plt.xlim(np.min(ks/pi),np.max(ks/pi))
plt.ylim(-1.5,1.5)
plt.yticks([-1.5,-1,-0.5,0,0.5,1,1.5])
plt.xticks([np.min(ks/pi),np.min(ks/pi)/2,0,np.max(ks/pi)/2,np.max(ks/pi)])
plt.ylabel(r"E[eV]/$\Delta$",fontdict={'weight':'normal','size': 15})
plt.xlabel(r"$k_{y}/\pi$",fontdict={'weight':'normal','size': 15})
plt.title(r"$\phi_{sc}$=0; Flux="+str(np.around(eta*(Lx*normal),4))+"$[h/e]$;",fontdict={'weight':'normal','size': 15})
plt.grid(ls='--',c='gray')
plt.show


plt.figure()
plt.plot(Phi_sc/pi, np.array(Andreev_TSC_G_Y),'b',lw=2)
plt.plot(Phi_sc/pi, np.array(Andreev_TSC_G_Y),'b',lw=2)
plt.xlim(np.min(Phi_sc/pi),np.max(Phi_sc/pi))
#plt.ylabel(r"E[eV]/$\Delta$",fontdict={'weight':'normal','size': 15})
plt.xlabel(r"$\phi_{sc}/\pi$",fontdict={'weight':'normal','size': 15})
pyplot.xticks(fontsize=15)
pyplot.yticks(fontsize=15)
plt.ylim(-1.,1.)
plt.yticks([-1.5,-1,-0.5,0,0.5,1,1.5])
# plt.xticks([0,0.5,1,1.5,2])
plt.title(r"$k_{y}$=0; theta="+str(np.around(np.arcsin((Lx//2)/25)/pi,3))+"r="+str(int(Lx//2)),\
           fontdict={'weight':'normal','size': 15})
plt.grid(ls='--',c='gray')
plt.show



Using matplotlib backend: TkAgg


NameError: name 'system_TSC_G_Y' is not defined